In [2]:
class MurmurHash3:
    def __init__(self, seed=0):
        self._len = 0  # the length of the current running byte sequence
        self.data = bytearray([0] * 4)  # the byte history for the last 4 bytes seen
        self._h1 = 0
        self._seed = seed;
        #self.reset()


    def reset(self):
        self._len = 0;
        self._h1 = self._seed;
        self.data = bytearray([0] * 4)


    def pushByte(self, b):
        self.data[self._len % 4] = b;
        self._len += 1;

        c1 = 0xcc9e2d51;
        c2 = 0x1b873593;


        h1_as_if_done = 0;  # We will use this as the value of self._h1 to dirty for returning to the caller
        if (self._len > 0 & self._len % 4 == 0):  # we have a valid history of 4 items!

            # little endian load order
            k1 = (self.data[0] & 0xff) | ((self.data[1] & 0xff) << 8) | ((self.data[2] & 0xff) << 16) | (
            self.data[3] << 24);
            k1 *= c1;
            
            #((k1 & 0xff) >> 17)
            k1 = (k1 << 15) | ((k1 & 0xff) >> 17);  # ROTL32(k1,15);
            k1 *= c2;
            self._h1 ^= k1;
            
            
            #((self._h1 & 0xff) >> 19)
            self._h1 = (self._h1 << 13) | ((self._h1 & 0xff) >> 19);  # ROTL32(h1,13);
            self._h1 = self._h1 * 5 + 0xe6546b64;
            h1_as_if_done = self._h1;

        else:  # tail case
            # tail
            k1 = 0;
            h1_as_if_done = self._h1;

            result = self._len & 0x03
            if result == 3:
                k1 = (self.data[2] & 0xff) << 16;
                # fallthrough
            elif result == 2:
                k1 |= (self.data[1] & 0xff) << 8;
                # fallthrough
            elif result == 1:
                k1 |= (self.data[0] & 0xff);
                k1 *= c1;
                #((k1 & 0xff) >> 17)
                k1 = (k1 << 15) | ((k1 & 0xff) >> 17);  # ROTL32(k1,15);
                k1 *= c2;
                h1_as_if_done ^= k1;


            # finalization
            h1_as_if_done ^= self._len;

            # fmix(h1);

            #((h1_as_if_done & 0xff) >> 16)

            h1_as_if_done ^= (h1_as_if_done & 0xff) >> 16;
            h1_as_if_done *= 0x85ebca6b;
            h1_as_if_done ^= (h1_as_if_done & 0xff) >> 13;
            h1_as_if_done *= 0xc2b2ae35;
            h1_as_if_done ^= (h1_as_if_done & 0xff) >> 16;
        print(h1_as_if_done);
        return h1_as_if_done;



In [23]:

import numpy as np


running_hash = MurmurHash3()
bytes = bytearray(range(10))
print("this is what I push in:", repr(bytes))
for i in range(len(bytes)):
    #print(i, bytes[i], type(bytes[i]))
    running_hash.pushByte(bytes[i])

('this is what I push in:', "bytearray(b'\\x00\\x01\\x02\\x03\\x04\\x05\\x06\\x07\\x08\\t')")
3864292196
544766536059655245979765107556
22592285068124815279694119802432356
925279229023134864899630211237946780516
37899437113808326596996351067994293198318436
1552360944181695930221160338007712339520493120356
63584704273682265193972634838350356757717607408397156
2604429487050025582345309819778292149918957265762773035876
106677431789569047852863890025801449638928650109542263831063396
4369507606100748200053304935456602686592443704285980339896266713956
